In [130]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
pd.set_option("display.float_format", lambda x: '%.2f' % x)

### 1- miuul_gezinomi.xlsx dosyasını okutunuz ve veri seti ile ilgili genel bilgileri gösteriniz..

In [131]:
df = pd.read_excel("../dataset/miuul_gezinomi.xlsx")
pd.DataFrame.set_index(df,"SaleId", inplace=True)

def check_df(dataframe, head=5):
    print("###Shape###")
    print(dataframe.shape)
    print("###Types###")
    print(dataframe.dtypes)
    print("###Head###")
    print(dataframe.head(head))
    print("###Tail###")
    print(dataframe.tail(head))
    print("###NA###")
    print(dataframe.isnull().sum())
    print("###Quantiles###")
    print(dataframe.describe([0,0.05,0.50,0.95,0.99,1]).T)

check_df(df)

###Shape###
(59164, 8)
###Types###
SaleDate              datetime64[ns]
CheckInDate           datetime64[ns]
Price                        float64
ConceptName                   object
SaleCityName                  object
CInDay                        object
SaleCheckInDayDiff             int64
Seasons                       object
dtype: object
###Head###
         SaleDate CheckInDate  Price     ConceptName SaleCityName    CInDay  SaleCheckInDayDiff Seasons
SaleId                                                                                                 
415122 2022-12-03  2022-12-03  79.30    Herşey Dahil      Antalya  Saturday                   0     Low
415103 2022-12-03  2022-12-03  45.97  Yarım Pansiyon      Antalya  Saturday                   0     Low
404034 2022-09-12  2022-09-13  77.84    Herşey Dahil      Antalya   Tuesday                   1    High
415094 2022-12-03  2022-12-10 222.71  Yarım Pansiyon        İzmir  Saturday                   7     Low
414951 2022-12-01  2

### 2- Kaç unique şehir vardır? Frekansları nedir?

In [132]:
df["SaleCityName"].nunique()

6

In [133]:
df["SaleCityName"].value_counts()

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

### 3- Kaç unique Concept vardır?

In [134]:
df["ConceptName"].nunique()

3

### 4- Hangi Concept’den kaçar tane satış gerçekleşmiş?

In [135]:
df["ConceptName"].value_counts()

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64

### 5- Şehirlere göre satışlardan toplam ne kadar kazanılmış?

In [136]:
#df.groupby("SaleCityName")["Price"].sum()
df.groupby("SaleCityName").agg({"Price": "sum"})

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


### 6- Concept türlerine göre göre ne kadar kazanılmış?

In [137]:
#df.groupby("ConceptName")["Price"].sum()
df.groupby("ConceptName").agg({"Price": "sum"})

,Price
ConceptName,
Herşey Dahil,3332910.77
Oda + Kahvaltı,121308.35
Yarım Pansiyon,174402.35


### 7- Şehirlere göre PRICE ortalamaları nedir?

In [138]:
#df.groupby("SaleCityName")["Price"].mean()
df.groupby("SaleCityName").agg({"Price": "mean"})

,Price
SaleCityName,
Antalya,64.52
Aydın,53.86
Diğer,47.71
Girne,59.48
Muğla,62.46
İzmir,66.27


### 8- Conceptlere göre PRICE ortalamaları nedir?

In [139]:
#df.groupby("ConceptName")["Price"].mean()
df.groupby("ConceptName").agg({"Price": "mean"})

,Price
ConceptName,
Herşey Dahil,62.67
Oda + Kahvaltı,50.25
Yarım Pansiyon,49.03


### 9- Şehir-Concept kırılımında PRICE ortalamaları nedir?

In [140]:
#df.groupby(["SaleCityName","ConceptName"])["Price"].mean()
df.groupby(["SaleCityName","ConceptName"]).agg({"Price": "mean"})

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil    64.52
             Oda + Kahvaltı  63.50
             Yarım Pansiyon  67.19
Aydın        Herşey Dahil    54.00
             Oda + Kahvaltı  34.46
             Yarım Pansiyon  30.02
Diğer        Herşey Dahil    84.77
             Oda + Kahvaltı  37.60
             Yarım Pansiyon  42.11
Girne        Herşey Dahil    97.68
             Oda + Kahvaltı  39.78
             Yarım Pansiyon  53.25
Muğla        Herşey Dahil    63.02
             Oda + Kahvaltı  59.04
             Yarım Pansiyon  45.12
İzmir        Herşey Dahil    74.70
             Oda + Kahvaltı  41.32
             Yarım Pansiyon  59.61

# Task 2

### SaleCheckInDayDiff değişkeni müşterinin CheckIn tarihinden ne kadar önce satin alımını tamamladığını gösterir.
### Aralıkları ikna edici şekilde oluşturunuz.
### Örneğin: ‘0_7’, ‘7_30', ‘30_90', ‘90_max’ aralıklarını kullanabilirsiniz.
### Bu aralıklar için "Last Minuters", "Potential Planners", "Planners", "Early Bookers“ isimlerini kullanabilirsiniz.

In [141]:
df["SaleCheckInDayDiff"]


SaleId
415122      0
415103      0
404034      1
415094      7
414951      2
         ... 
51817     279
51816     279
51814       1
51736       1
51731     231
Name: SaleCheckInDayDiff, Length: 59164, dtype: int64

In [142]:
bins = [-1,7,30,90, df["SaleCheckInDayDiff"].max()]
categories = ["Last Minute", "Potential Planners", "Planners", "Early Bookers"]

df["EB_score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels= categories)
df.head(50).to_excel("/Users/umut/Desktop/DataProjects/PotentialCustomerYield/dataset/eb_score.xlsx", index=False)


# Task 3

### Şehir-Concept-EB Score, Şehir-Concept- Sezon, Şehir-Concept-CInDay kırılımında ortalama ödenen ücret ve yapılan işlem sayısı cinsinden inceleyiniz ?

In [143]:
df.groupby(["SaleCityName","ConceptName","EB_score"]).agg({"Price": ["mean","count"]})

Price       
                                                mean  count
SaleCityName ConceptName    EB_score                       
Antalya      Herşey Dahil   Last Minute        62.75  14148
                            Potential Planners 64.90   8874
                            Planners           67.88   4490
                            Early Bookers      66.49   3281
             Oda + Kahvaltı Last Minute        65.35    503
...                                              ...    ...
İzmir        Oda + Kahvaltı Early Bookers      66.93      3
             Yarım Pansiyon Last Minute        57.79    636
                            Potential Planners 64.80    198
                            Planners           60.51     47
                            Early Bookers      65.91     14

[72 rows x 2 columns]

In [144]:
df.groupby(["SaleCityName","ConceptName","Seasons"]).agg({"Price": ["mean","count"]})

Price       
                                      mean  count
SaleCityName ConceptName    Seasons              
Antalya      Herşey Dahil   High     64.92  27126
                            Low      61.55   3667
             Oda + Kahvaltı High     66.27    303
                            Low      60.67    295
             Yarım Pansiyon High     73.26    118
                            Low      62.00    138
Aydın        Herşey Dahil   High     54.95  10103
                            Low      33.68    473
             Oda + Kahvaltı High     30.39     27
                            Low      44.45     11
             Yarım Pansiyon High     32.63     20
                            Low      25.27     11
Diğer        Herşey Dahil   High     83.79    364
                            Low      87.31    141
             Oda + Kahvaltı High     34.84    349
                            Low      39.96    408
             Yarım Pansiyon High     39.73    516
                            Low      42.95   1462
Girne        Herşey Dahil   High    103.94     55
                            Low      90.94     51
             Oda + Kahvaltı High     39.48     66
                            Low      40.04     73
             Yarım Pansiyon High     58.38    100
                            Low      48.58    110
Muğla        Herşey Dahil   High     63.38   9582
                            Low      48.30    237
             Oda + Kahvaltı High     61.26    592
                            Low      37.77     62
             Yarım Pansiyon High     45.82    177
                            Low      32.68     10
İzmir        Herşey Dahil   High     74.75   1234
                            Low      74.31    147
             Oda + Kahvaltı High     54.14     86
                            Low      33.56    142
             Yarım Pansiyon High     87.66    118
                            Low      55.35    777

In [145]:
df.groupby(["SaleCityName","ConceptName","CInDay"]).agg({"Price": ["mean","count"]})

Price      
                                       mean count
SaleCityName ConceptName    CInDay               
Antalya      Herşey Dahil   Friday    62.66  4136
                            Monday    63.26  6831
                            Saturday  64.42  4741
                            Sunday    65.85  3818
                            Thursday  62.89  3898
...                                     ...   ...
İzmir        Yarım Pansiyon Saturday  52.50   315
                            Sunday    98.93    68
                            Thursday  52.48    91
                            Tuesday   56.14    73
                            Wednesday 54.06    86

[126 rows x 2 columns]

### Elde ettiğiniz çıktıyı agg_df olarak kaydediniz.

In [168]:
agg_df = df.groupby(["SaleCityName","ConceptName","Seasons"]).agg({"Price": "mean"})
agg_df.head(18)

Price
SaleCityName ConceptName    Seasons       
Antalya      Herşey Dahil   High     64.92
                            Low      61.55
             Oda + Kahvaltı High     66.27
                            Low      60.67
             Yarım Pansiyon High     73.26
                            Low      62.00
Aydın        Herşey Dahil   High     54.95
                            Low      33.68
             Oda + Kahvaltı High     30.39
                            Low      44.45
             Yarım Pansiyon High     32.63
                            Low      25.27
Diğer        Herşey Dahil   High     83.79
                            Low      87.31
             Oda + Kahvaltı High     34.84
                            Low      39.96
             Yarım Pansiyon High     39.73
                            Low      42.95

### Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir. Bu isimleri değişken isimlerine çeviriniz.

In [169]:
agg_df.reset_index(inplace=True)

In [170]:
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Antalya,Herşey Dahil,High,64.92
1,Antalya,Herşey Dahil,Low,61.55
2,Antalya,Oda + Kahvaltı,High,66.27
3,Antalya,Oda + Kahvaltı,Low,60.67
4,Antalya,Yarım Pansiyon,High,73.26


### • Yeni seviye tabanlı satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.
### • Yeni eklenecek değişkenin adı: sales_level_based
### • Önceki soruda elde edeceğiniz çıktıdaki gözlemleri bir araya getirerek sales_level_based değişkenini oluşturmanız gerekmektedir.

In [177]:
agg_df["sales_level_based"] = agg_df[["SaleCityName","ConceptName","Seasons"]].agg(lambda x: '_'.join(x).upper(), axis=1)

In [178]:
agg_df.head(10)

,SaleCityName,ConceptName,Seasons,Price,SEGMENT,sales_level_based
0,Antalya,Herşey Dahil,High,64.92,B,ANTALYA_HERŞEY DAHIL_HIGH
1,Antalya,Herşey Dahil,Low,61.55,B,ANTALYA_HERŞEY DAHIL_LOW
2,Antalya,Oda + Kahvaltı,High,66.27,A,ANTALYA_ODA + KAHVALTI_HIGH
3,Antalya,Oda + Kahvaltı,Low,60.67,B,ANTALYA_ODA + KAHVALTI_LOW
4,Antalya,Yarım Pansiyon,High,73.26,A,ANTALYA_YARIM PANSIYON_HIGH
5,Antalya,Yarım Pansiyon,Low,62.00,B,ANTALYA_YARIM PANSIYON_LOW
6,Aydın,Herşey Dahil,High,54.95,B,AYDIN_HERŞEY DAHIL_HIGH
7,Aydın,Herşey Dahil,Low,33.68,D,AYDIN_HERŞEY DAHIL_LOW
8,Aydın,Oda + Kahvaltı,High,30.39,D,AYDIN_ODA + KAHVALTI_HIGH
9,Aydın,Oda + Kahvaltı,Low,44.45,C,AYDIN_ODA + KAHVALTI_LOW


### • Yeni personaları PRICE’a göre 4 segmente ayırınız.
### • Segmentleri SEGMENT isimlendirmesi ile değişken olarak agg_df’e ekleyiniz.
### • Segmentleri betimleyiniz (Segmentlere göre group by yapıp price mean, max, sum’larını alınız).

In [179]:
agg_df["Price"].dtypes

dtype('float64')

In [180]:
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.head(30)

,SaleCityName,ConceptName,Seasons,Price,SEGMENT,sales_level_based
0,Antalya,Herşey Dahil,High,64.92,B,ANTALYA_HERŞEY DAHIL_HIGH
1,Antalya,Herşey Dahil,Low,61.55,B,ANTALYA_HERŞEY DAHIL_LOW
2,Antalya,Oda + Kahvaltı,High,66.27,A,ANTALYA_ODA + KAHVALTI_HIGH
3,Antalya,Oda + Kahvaltı,Low,60.67,B,ANTALYA_ODA + KAHVALTI_LOW
4,Antalya,Yarım Pansiyon,High,73.26,A,ANTALYA_YARIM PANSIYON_HIGH
5,Antalya,Yarım Pansiyon,Low,62.00,B,ANTALYA_YARIM PANSIYON_LOW
6,Aydın,Herşey Dahil,High,54.95,B,AYDIN_HERŞEY DAHIL_HIGH
7,Aydın,Herşey Dahil,Low,33.68,D,AYDIN_HERŞEY DAHIL_LOW
8,Aydın,Oda + Kahvaltı,High,30.39,D,AYDIN_ODA + KAHVALTI_HIGH
9,Aydın,Oda + Kahvaltı,Low,44.45,C,AYDIN_ODA + KAHVALTI_LOW


In [181]:
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})

Price              
         mean    max    sum
SEGMENT                    
D       33.37  39.48 300.30
C       44.89  54.14 403.99
B       60.27  64.92 542.47
A       82.47 103.94 742.21

### • Antalya’da herşey dahil ve yüksek sezonda tatil yapmak isteyen bir kişinin ortalama ne kadar gelir kazandırması beklenir?
### • Girne’de yarım pansiyon bir otele düşük sezonda giden bir tatilci hangi segmentte yer alacaktır?

In [191]:
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
agg_df[agg_df["sales_level_based"] == new_user]["Price"]



0   64.92
Name: Price, dtype: float64


In [192]:
new_user2 = "GIRNE_YARIM PANSIYON_LOW"
agg_df[agg_df["sales_level_based"] == new_user]["SEGMENT"]

0    B
Name: SEGMENT, dtype: category
Categories (4, object): ['D' < 'C' < 'B' < 'A']